In [12]:
#!pip install scikit-learn
#!pip install casatools casatasks


In [1]:
import scipy
import pylab
import os
import gc
import pprint
import numpy as np
import pandas as pd
import csv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import astropy.io.fits as fits
from astropy import units as u
from astropy.wcs import WCS
import pyspeckit
import analysis as an
import pyneb as pn
from spectral_cube import SpectralCube 
import sklearn
from sklearn.decomposition import PCA
from reproject import reproject_interp
from astropy.convolution import convolve, Gaussian2DKernel



%matplotlib inline
                                            # Suppress warnings we don't care about:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

                                            # LaTeX:
matplotlib.rc('text', usetex=True)
#matplotlib.rc('font', family='sans-serif')
font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)
#if not working: in terminal where launch jupyter notebook run
#export PATH=$PATH:/Library/TeX/texbin/latex

## CASA imports

In [9]:
from casatools import image, table
from casatasks import importfits, imstat, tclean, exportfits
from casatools import image, regionmanager
from casatasks import imcontsub

import os
import csv
import shutil
from glob import glob

# CASA tasks (assumed to be available in your environment)
from casatasks import importfits, imcontsub, exportfits

data_update using version casarundata-2025.12.12-1.tar.gz, acquiring the lock ... 


lock file is not empty : /Users/u826s018/.casa/data/data_update.lock
data_update: the lock file at /Users/u826s018/.casa/data is not empty.
A previous attempt to update path may have failed or exited prematurely.
Remove the lock file and set force to True with the desired version (default to most recent).
It may be best to completely re-populate path using pull_data and measures_update.
2026-02-06 17:55:40	SEVERE	::casa	lock file is not empty : /Users/u826s018/.casa/data/data_update.lock
2026-02-06 17:55:40	SEVERE	::casa	data_update: the lock file at /Users/u826s018/.casa/data is not empty.
2026-02-06 17:55:40	SEVERE	::casa	A previous attempt to update path may have failed or exited prematurely.
2026-02-06 17:55:40	SEVERE	::casa	Remove the lock file and set force to True with the desired version (default to most recent).
2026-02-06 17:55:40	SEVERE	::casa	It may be best to completely re-populate path using pull_data and measures_update.
2026-02-06 17:55:40	SEVERE	::casa	
2026-02-06 17:5

In [11]:
# This block creates the initial parameter CSV for continuum subtraction.
# It only generates entries for files that do NOT already have a contsub
# parameter entry in the folder. This is meant for the first manual pass.
#
# Workflow:
# 1. First, run this cell to generate a new parameter CSV containing the
#    channel values you determined manually for each line/cube.
#
# 2. After inspecting the individual channels and updating the values in
#    this new CSV, you should merge it into the main parameter file using
#    the merge script in the next code block.
#
# 3. Once the contsub file exists, you normally do NOT rerun this cell.
#    Instead, you simply edit the CSV directly and run the autolooping code.
#
# 4. If you ever need to rerun the workflow for a single line without
#    overwriting everything, you can delete that one entry, regenerate it
#    with this cell, update it manually, and then merge it again.
#
# Requirements:
# - The input_dir must contain the line subcubes.
# - CASA will write the continuum-subtracted outputs to output_dir.


input_dir = "../NGC253_output/3channel/atomic"
output_dir = "../NGC253_output/3channel/casa/atomic"
os.makedirs(output_dir, exist_ok=True)

log_file = os.path.join(output_dir, "imcontsub_params.csv")

# Load existing processed line names
processed_lines = set()
if os.path.exists(log_file):
    with open(log_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        line_key = "line_name" if "line_name" in header else header[0]  # fallback to first column
        for row in reader:
            processed_lines.add(row[line_key])


# Prepare to collect new entries
new_entries = []

fits_files = glob(os.path.join(input_dir, "*.fits"))
for fits_path in fits_files:
    line_name = os.path.splitext(os.path.basename(fits_path))[0]
    casa_image_path = os.path.join(output_dir, f"{line_name}.im")
    linefile = os.path.join(output_dir, f"{line_name}.line")
    contfile = os.path.join(output_dir, f"{line_name}.cont")

    # Skip if already processed or output files exist
    if line_name in processed_lines or (os.path.exists(linefile) and os.path.exists(contfile)):
        print(f"Skipping {line_name} (already processed or output files exist)")
        continue

    # Import FITS to CASA
    importfits(
        fitsimage=fits_path,
        imagename=casa_image_path,
        whichhdu=0,
        whichrep=0,
        zeroblanks=True,
        overwrite=True,
        defaultaxes=False,
        defaultaxesvalues=[],
        beam=[]
    )

    # Ask user for parameters
    chans = input(f"Enter chans value for {line_name}: ")
    fitorder = int(input(f"Enter fitorder for {line_name} (e.g., 0, 1, 2): "))

    # Run imcontsub
    imcontsub(
        imagename=casa_image_path,
        linefile=linefile,
        contfile=contfile,
        fitorder=fitorder,
        region='',
        box='',
        chans=chans,
        stokes=''
    )

    # Export to FITS
    exportfits(imagename=linefile, fitsimage=linefile + ".fits", overwrite=True)
    exportfits(imagename=contfile, fitsimage=contfile + ".fits", overwrite=True)

    # Store new entry
    new_entries.append([line_name, fitorder, chans, casa_image_path, linefile, contfile])

# Append new entries to CSV, writing header only if needed
write_header = not os.path.exists(log_file) or os.path.getsize(log_file) == 0

with open(log_file, "a", newline="") as f:
    writer = csv.writer(f)
    if write_header:
        writer.writerow(["line_name", "fitorder", "chans", "imagename", "linefile", "contfile"])
    for entry in new_entries:
        writer.writerow(entry)


print(f"Appended {len(new_entries)} new entries to:", log_file)


Skipping NGC253_CO2_15.0 (already processed or output files exist)
Skipping NGC253_MgV_5.6 (already processed or output files exist)
Skipping NGC253_H5b_7.5 (already processed or output files exist)
Skipping NGC253_H6d_5.1 (already processed or output files exist)
Skipping NGC253_H7z_6.3 (already processed or output files exist)
Skipping NGC253_ArII_7.0 (already processed or output files exist)
Skipping NGC253_He_6.7 (already processed or output files exist)
Skipping NGC253_H7k_5.4 (already processed or output files exist)
Skipping NGC253_MgV_13.5 (already processed or output files exist)
Skipping NGC253_H6g_5.9 (already processed or output files exist)
Skipping NGC253_NeIII_15.6 (already processed or output files exist)
Skipping NGC253_H7e_6.8 (already processed or output files exist)
Skipping NGC253_NeII_12.8 (already processed or output files exist)
Skipping NGC253_H6a_12.4 (already processed or output files exist)
Skipping NGC253_H8b_16.2 (already processed or output files exist)
S

In [20]:
'''
# This script is used to update the main parameter file used by the autolooping code.
# The workflow is:
# 1. For each new line or cube, we first inspect the individual channels manually
#    and record the correct channel values in a separate parameter file (new_file).
#
# 2. The autolooping code always reads from the main parameter file (main_file),
#    so before running the autoloop we merge the updated entries from new_file
#    into main_file.
#
# 3. This script performs that merge: it combines both files, keeps the most recent
#    entry for each line_name, and writes the updated result back to main_file.
#
# 4. If we need to rerun the workflow for only one specific line without modifying
#    the main parameter file, we can create a temporary parameter file, update only
#    that line there, and then run this merge script to safely update main_file.
#
# This keeps the main parameter file clean and ensures that autolooping always uses
# the latest manually verified channel values.

import pandas as pd

main_file = "../NGC253_output/3channel/imcontsub_params.csv"
new_file = "../NGC253_output/3channel/casa/atomic/imcontsub_params.csv"

df_main = pd.read_csv(main_file)
df_new = pd.read_csv(new_file)

df_new.columns = df_main.columns  # ensure column match

df_merged = pd.concat([df_main, df_new], ignore_index=True)
df_merged.drop_duplicates(subset="line_name", keep="last", inplace=True)  # keep updated rows

df_merged.to_csv(main_file, index=False)

print("✅ Merged and updated with replacements:", main_file)
'''

'\nimport pandas as pd\n\nmain_file = "../NGC253_output/3channel/imcontsub_params.csv"\nnew_file = "../NGC253_output/3channel/casa/atomic/imcontsub_params.csv"\n\ndf_main = pd.read_csv(main_file)\ndf_new = pd.read_csv(new_file)\n\ndf_new.columns = df_main.columns  # ensure column match\n\ndf_merged = pd.concat([df_main, df_new], ignore_index=True)\ndf_merged.drop_duplicates(subset="line_name", keep="last", inplace=True)  # keep updated rows\n\ndf_merged.to_csv(main_file, index=False)\n\nprint("✅ Merged and updated with replacements:", main_file)\n'

### Autolooping code

In [12]:
#Advanced form of autolooping code should be able to handle errors or any issues
# if doesnot work go to last to find initial version to tackle problem
#Will use the parameter file csv and run all deleting fits file before creating line and continuum automatically


# 📁 Define directories
input_dir = "../NGC253_output/3channel/atomic"
output_dir = "../NGC253_output/3channel/casa/atomic"
os.makedirs(output_dir, exist_ok=True)

log_file = os.path.join(output_dir, "imcontsub_params.csv")
param_file = os.path.join(input_dir, "imcontsub_params.csv")

# ⚠️ Clean output_dir before starting
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# 📄 Load parameters from CSV into a dictionary
params = {}
with open(param_file, "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        line_name = row["line_name"].strip()
        chans = row["chans"].strip()
        fitorder_str = row["fitorder"].strip()

        if fitorder_str in ("1", "2"):
            fitorder = int(fitorder_str)
            params[line_name] = {"chans": chans, "fitorder": fitorder}
        else:
            print(f"⚠️ Skipping '{line_name}': invalid or missing fitorder '{fitorder_str}'")

# 📝 Initialize CSV logging if it doesn't exist
if not os.path.exists(log_file):
    with open(log_file, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["line_name", "fitorder", "chans", "imagename", "linefile", "contfile"])

# 🔄 Loop through all FITS files in input directory
fits_files = glob(os.path.join(input_dir, "*.fits"))

for fits_path in fits_files:
    line_name = os.path.splitext(os.path.basename(fits_path))[0]

    if line_name not in params:
        print(f"⚠️ Skipping {line_name}: no parameters found in {param_file}")
        continue

    chans = params[line_name]["chans"]
    fitorder = params[line_name]["fitorder"]

    casa_image_path = os.path.join(output_dir, f"{line_name}.im")
    linefile = os.path.join(output_dir, f"{line_name}.line")
    contfile = os.path.join(output_dir, f"{line_name}.cont")

    # 📥 Import FITS to CASA
    importfits(
        fitsimage=fits_path,
        imagename=casa_image_path,
        whichhdu=0,
        whichrep=0,
        zeroblanks=True,
        overwrite=True,
        defaultaxes=False,
        defaultaxesvalues=[],
        beam=[]
    )

    # 🧪 Run imcontsub
    imcontsub(
        imagename=casa_image_path,
        linefile=linefile,
        contfile=contfile,
        fitorder=fitorder,
        region='',
        box='',
        chans=chans,
        stokes=''
    )

    # 📤 Export line and continuum to FITS
    exportfits(imagename=linefile, fitsimage=linefile + ".fits", overwrite=True)
    exportfits(imagename=contfile, fitsimage=contfile + ".fits", overwrite=True)

    # 🧾 Record parameters into run log
    with open(log_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([line_name, fitorder, chans, casa_image_path, linefile, contfile])

print("✅ All files processed. Run log saved to:", log_file)


2026-02-06 17:56:59	WARN	importfits::::casa	This image has no beam or angular resolution provided, so you will not receive warnings from
2026-02-06 17:56:59	WARN	importfits::::casa+	tasks such as imregrid if your image pixels do not sample the the angular resolution well.
2026-02-06 17:56:59	WARN	importfits::::casa+	(This only affects warnings, not any functionality).
2026-02-06 17:56:59	WARN	importfits::::casa+	Providing a beam and brightness units in an image can also be useful for flux calculations.
2026-02-06 17:56:59	WARN	importfits::::casa+	If you wish to add a beam or brightness units to your image, please use
2026-02-06 17:56:59	WARN	importfits::::casa+	the "beam" parameter or ia.setrestoringbeam() and ia.setbrightnessunit()
2026-02-06 17:56:59	WARN	importfits::::casa	This image has no beam or angular resolution provided, so you will not receive warnings from
2026-02-06 17:56:59	WARN	importfits::::casa+	tasks such as imregrid if your image pixels do not sample the the angular r

✅ All files processed. Run log saved to: ../NGC253_output/3channel/casa/atomic/imcontsub_params.csv


## Smoothing and regriding the line file

In [13]:

# ---------- USER PARAMETERS ----------
IN_DIR  = "../NGC253_output/3channel/casa/atomic"  # folder containing MIRI .fits cubes
OUT_DIR = "../NGC253_output/3channel/casa/line_smooth"   # output folder
REF_CUBE = "../NGC253_output/3channel/casa/atomic/NGC253_PIII_17.9.line.fits" # Fe reference
TARGET_LAMBDA = 17.8846  # µm
FILE_GLOB = os.path.join(IN_DIR, "*.fits")
# ------------------------------------

os.makedirs(OUT_DIR, exist_ok=True)
files = sorted(glob(FILE_GLOB))

if len(files) == 0:
    raise RuntimeError(f"No FITS cubes found in {IN_DIR}")

# --- Load reference cube wavelength grid ---
with fits.open(REF_CUBE) as ref:
    hdr_ref = ref[0].header
    naxis3 = hdr_ref["NAXIS3"]
    lam_ref = np.arange(naxis3) * hdr_ref["CDELT3"] + hdr_ref["CRVAL3"]

target_fwhm = 0.033 * TARGET_LAMBDA + 0.106  # arcsec

for infile in files:
    print(f"Processing: {os.path.basename(infile)}")
    sc = SpectralCube.read(infile, hdu=1)
    data = fits.getdata(infile, hdu=1)
    hdr = fits.getheader(infile, hdu=1)

    n_spec = hdr["NAXIS3"]
    lam = np.arange(n_spec) * hdr["CDELT3"] + hdr["CRVAL3"]

    pixscale = abs(hdr["CDELT2"]) * 3600.0  # arcsec/pix
    smoothed = np.zeros_like(data)

    mask = np.ones_like(data[0])
    mask[np.isnan(data[0])] = np.nan

    for i, lam_i in enumerate(lam):
        theta = 0.033 * lam_i + 0.106  # arcsec
        fwhm = np.sqrt(np.clip(target_fwhm**2 - theta**2, 0, None))
        sigma_pix = fwhm / 2.354 / pixscale

        if sigma_pix > 0:
            kernel = Gaussian2DKernel(x_stddev=sigma_pix)
            smoothed[i] = convolve(data[i], kernel)
        else:
            smoothed[i] = data[i]

    outname = os.path.join(OUT_DIR, os.path.basename(infile).replace(".fits", "_smooth.fits"))
    fits.writeto(outname, smoothed * mask, hdr, overwrite=True)
    print(f" → Saved: {outname}")

print("All cubes smoothed successfully.")


Processing: NGC253_ArIII_9.0.cont.fits
 → Saved: ../NGC253_output/3channel/casa/line_smooth/NGC253_ArIII_9.0.cont_smooth.fits
Processing: NGC253_ArIII_9.0.line.fits
 → Saved: ../NGC253_output/3channel/casa/line_smooth/NGC253_ArIII_9.0.line_smooth.fits
Processing: NGC253_ArII_7.0.cont.fits
 → Saved: ../NGC253_output/3channel/casa/line_smooth/NGC253_ArII_7.0.cont_smooth.fits
Processing: NGC253_ArII_7.0.line.fits
 → Saved: ../NGC253_output/3channel/casa/line_smooth/NGC253_ArII_7.0.line_smooth.fits
Processing: NGC253_C2H2_13.7.cont.fits
 → Saved: ../NGC253_output/3channel/casa/line_smooth/NGC253_C2H2_13.7.cont_smooth.fits
Processing: NGC253_C2H2_13.7.line.fits
 → Saved: ../NGC253_output/3channel/casa/line_smooth/NGC253_C2H2_13.7.line_smooth.fits
Processing: NGC253_CO2_15.0.cont.fits
 → Saved: ../NGC253_output/3channel/casa/line_smooth/NGC253_CO2_15.0.cont_smooth.fits
Processing: NGC253_CO2_15.0.line.fits
 → Saved: ../NGC253_output/3channel/casa/line_smooth/NGC253_CO2_15.0.line_smooth.fits


In [14]:
## Trying to correctly pass the header:


# --- paths ---
smooth_dir = "../NGC253_output/3channel/casa/line_smooth"
out_dir = "../NGC253_output/3channel/casa/line_regrid"
os.makedirs(out_dir, exist_ok=True)

# Reference cube (Fe II or ch4)
ref_path = os.path.join(smooth_dir, "NGC253_PIII_17.9.line_smooth.fits")
with fits.open(ref_path) as ref_hdul:
    ref_hdr = ref_hdul[0].header.copy()
    ref_data = ref_hdul[0].data.copy()
ref_wcs_3d = WCS(ref_hdr)
ref_wcs_2d = ref_wcs_3d.dropaxis(2)
ref_hdr_2d = ref_wcs_2d.to_header()
ny_ref, nx_ref = ref_data.shape[1], ref_data.shape[2]
shape_out_ref = (ny_ref, nx_ref)

# --- Input cubes ---
cube_files = sorted(glob(os.path.join(smooth_dir, "*.fits")))
print(f"Found {len(cube_files)} smoothed cubes.")

for infile in cube_files:
    name = os.path.basename(infile)
    outfile = os.path.join(out_dir, name.replace(".fits", "_regrid.fits"))

    # --- Read input cube ---
    data_in = fits.getdata(infile)
    hdr_in = fits.getheader(infile)
    wcs_in_3d = WCS(hdr_in)
    wcs_in_2d = wcs_in_3d.dropaxis(2)
    hdr_in_2d = wcs_in_2d.to_header()

    nlam = data_in.shape[0]
    print(f"Regridding {name}: {nlam} slices -> {shape_out_ref}")

    # --- Reproject slice by slice ---
    regridded = np.empty((nlam, ny_ref, nx_ref), dtype=np.float32)
    for i in range(nlam):
        plane = data_in[i].astype(float)
        plane_masked = np.where(np.isfinite(plane), plane, np.nan)
        out_plane, _ = reproject_interp((plane_masked, hdr_in_2d),
                                        ref_hdr_2d, shape_out=shape_out_ref)
        regridded[i] = out_plane

    # --- Build output header ---
    out_hdr = ref_hdr.copy()
    out_hdr["NAXIS3"] = nlam
    for key in hdr_in:
        if "3" in key or key.startswith("REST") or key.startswith("SPECSYS"):
            out_hdr[key] = hdr_in[key]



    fits.writeto(outfile, regridded, out_hdr, overwrite=True)
    print(f"  → wrote {outfile} shape={regridded.shape}")

print("All smoothed cubes regridded successfully.")



Found 100 smoothed cubes.
Regridding NGC253_ArIII_9.0.cont_smooth.fits: 53 slices -> (93, 75)
  → wrote ../NGC253_output/3channel/casa/line_regrid/NGC253_ArIII_9.0.cont_smooth_regrid.fits shape=(53, 93, 75)
Regridding NGC253_ArIII_9.0.line_smooth.fits: 53 slices -> (93, 75)
  → wrote ../NGC253_output/3channel/casa/line_regrid/NGC253_ArIII_9.0.line_smooth_regrid.fits shape=(53, 93, 75)
Regridding NGC253_ArII_7.0.cont_smooth.fits: 67 slices -> (93, 75)
  → wrote ../NGC253_output/3channel/casa/line_regrid/NGC253_ArII_7.0.cont_smooth_regrid.fits shape=(67, 93, 75)
Regridding NGC253_ArII_7.0.line_smooth.fits: 67 slices -> (93, 75)
  → wrote ../NGC253_output/3channel/casa/line_regrid/NGC253_ArII_7.0.line_smooth_regrid.fits shape=(67, 93, 75)
Regridding NGC253_C2H2_13.7.cont_smooth.fits: 42 slices -> (93, 75)
  → wrote ../NGC253_output/3channel/casa/line_regrid/NGC253_C2H2_13.7.cont_smooth_regrid.fits shape=(42, 93, 75)
Regridding NGC253_C2H2_13.7.line_smooth.fits: 42 slices -> (93, 75)
  → w

In [15]:

# ---- Paths ----
input_dir = "../NGC253_output/3channel/casa/line_regrid"          #use this for frequency based x axis
output_dir = "../NGC253_output/3channel/casa/line_masked"

os.makedirs(output_dir, exist_ok=True)

# ---- Reference Ch I cube for mask ----
ch1_ref = os.path.join(input_dir, "NGC253_S9_5.0.line_smooth_regrid.fits")         #for frequency based maps
if not os.path.exists(ch1_ref):
    raise FileNotFoundError(f"Ch 1 reference cube not found: {ch1_ref}")

fe_data = fits.getdata(ch1_ref)
if fe_data.ndim != 3:
    raise RuntimeError(f"Ch I cube not 3D: shape={fe_data.shape}")

mask = np.ones_like(fe_data[0], dtype=float)
mask[~np.isfinite(fe_data[0])] = np.nan
print(f"✅ Mask built from Ch I first plane: shape={mask.shape}")

# ---- Apply mask to each cube ----
for fname in sorted(os.listdir(input_dir)):
    if not fname.endswith(".fits"):
        continue
    infile = os.path.join(input_dir, fname)
    data = fits.getdata(infile)
    hdr = fits.getheader(infile)

    if data.ndim != 3:
        print(f"Skipping {fname} (not 3D)")
        continue

    # Apply mask: NaN where Ch I is invalid
    data_masked = data * mask[np.newaxis, :, :]

    # Write masked cube
    outfile = os.path.join(output_dir, fname.replace(".fits", "_maskch1.fits"))
    fits.writeto(outfile, data_masked, hdr, overwrite=True)
    print(f"→ Masked {fname}  →  {os.path.basename(outfile)}")

print("🎉 All cubes masked using ch1 S9 reference.")


✅ Mask built from Ch I first plane: shape=(93, 75)
→ Masked NGC253_ArIII_9.0.cont_smooth_regrid.fits  →  NGC253_ArIII_9.0.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_ArIII_9.0.line_smooth_regrid.fits  →  NGC253_ArIII_9.0.line_smooth_regrid_maskch1.fits
→ Masked NGC253_ArII_7.0.cont_smooth_regrid.fits  →  NGC253_ArII_7.0.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_ArII_7.0.line_smooth_regrid.fits  →  NGC253_ArII_7.0.line_smooth_regrid_maskch1.fits
→ Masked NGC253_C2H2_13.7.cont_smooth_regrid.fits  →  NGC253_C2H2_13.7.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_C2H2_13.7.line_smooth_regrid.fits  →  NGC253_C2H2_13.7.line_smooth_regrid_maskch1.fits
→ Masked NGC253_CO2_15.0.cont_smooth_regrid.fits  →  NGC253_CO2_15.0.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_CO2_15.0.line_smooth_regrid.fits  →  NGC253_CO2_15.0.line_smooth_regrid_maskch1.fits
→ Masked NGC253_ClII_14.4.cont_smooth_regrid.fits  →  NGC253_ClII_14.4.cont_smooth_regrid_maskch1.fits
→ Masked NGC253_ClII_14.4.line

### Moment maps

## Code I might require for checks/ error in workflow, these may contain my earlier version of codes

### Sanity Check